# Importer les librairies, et fichiers .py auxiliaires

In [5]:
import numpy as np
from preprocess_data import *
from bayes_classifier import BayesClassifier
from complement_naive_bayes import train_cnb_with_tfidf, train_cnb
from ensemble_learning import train_ensemble

from save_output import save_output

# Exploration de données

# Jalon 1) Naive de Bayes vanille
K-Fold Validation croisée ($k = 7)$:
- α: Lissage du postérieur de Bayes

In [ ]:
data_preprocess = DataPreprocess()
bayes_classifier = BayesClassifier()
best_lps, best_f1 = bayes_classifier.hyperparameter_tuning(data_preprocess.train, data_preprocess.label_train)
print(best_lps, best_f1)
tuned_bayes_classifier = BayesClassifier()
tuned_bayes_classifier.fit(data_preprocess.train, data_preprocess.label_train, best_lps)
predictions = np.array([tuned_bayes_classifier.predict(x_i) for x_i in data_preprocess.test])
save_output(predictions, "bayes", best_lps, "stopwords")

# Jalon 2)

Naive Bayes Complement (avec TFIDF)

In [ ]:
data_preprocess = DataPreprocess()
complement_naive_bayes, tfidf_transformer = train_cnb_with_tfidf(data_preprocess.train, data_preprocess.label_train)
tfidf_test = tfidf_transformer.transform(data_preprocess.test)
predictions = complement_naive_bayes.predict(tfidf_test)
save_output(predictions, "cnb", "random_search_50_iter", "tfidf")

Naive Bayes Complement

In [ ]:
data_preprocess = DataPreprocess()
complement_naive_bayes = train_cnb(data_preprocess.train, data_preprocess.label_train)
predictions = complement_naive_bayes.predict(data_preprocess.test)
save_output(predictions, "cnb", "random_search_50_iter", "no_preprocessing")

XGBoost

# Apprentissage par Ensemble

CNB, XGBoost, Logistic Regression sans stopwords, sous-échantillonage

In [6]:
data_preprocess = DataPreprocess()
data_preprocess.remove_stopwords()
X_train_undersampled, y_train_undersampled = random_undersampling(data_preprocess.train, data_preprocess.label_train)
model_names = ['ComplementNB', 'XGBoost', 'LogisticRegression']
best_ensemble_model = train_ensemble(X_train_undersampled, y_train_undersampled, model_names)
predictions_voter = best_ensemble_model.predict(data_preprocess.test)
save_output(predictions_voter, "ensemble_cnb_xgboost_logreg", "random_search_15_iter", "stopwords")

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END ...model__alpha=0.3851694527491273;, score=0.687 total time=   3.4s
[CV 2/2] END ...model__alpha=0.3851694527491273;, score=0.683 total time=   4.0s
Best F1 for ComplementNB: 0.6849461126344383
Confusion Matrix - Fold 1:
[[1298  483]
 [ 296  853]]

Confusion Matrix - Fold 2:
[[1275  506]
 [ 290  859]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.628 total time=  53.7s
[CV 2/2] END model__learning_rate=0.11976270078546496, model__max_depth=5, model__n_estimators=200, model__subsample=0.8411053504286575;, score=0.634 total time=  54.3s
Best F1 for XGBClassifier: 0.6313404243152057
Confusion Matrix - Fold 1:
[[1498  283]
 [ 493  656]]

Confusion Matrix - Fold 2:
[[1466  315]
 [ 469  680]]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END

ValueError: invalid literal for int() with base 10: '[0]'